In [1]:
import sys;
sys.path.insert(0, '..')

## Chapter 11 Code Snippets and Listings

### Finding desired outcomes with the Grover Adaptive Search method (11.1)

We can use the list of tuples to encode the function $f(k) = k^2 -5 $ as a polynomial of $n = 2$ binary variables:

In [2]:
terms = [(4, [1]), (4, [1, 0]), (1, [0]), (-5, [])]

Now we can encode our polynomial of binary variables using the function `build_polynomial_circuit` from chapter 10:

In [3]:
from sim_circuit import *
from algo import build_polynomial_circuit

n_key = 2
n_value = 4

qc = build_polynomial_circuit(n_key, n_value, terms)

In [4]:
from util import grid_state

grid_state(qc.run(), n_key, neg = True, show_probs = False)



╒═══════════╤══════════╤══════════╤══════════╤══════════╕
│           │ 0 = 00   │ 1 = 01   │ 2 = 10   │ 3 = 11   │
╞═══════════╪══════════╪══════════╪══════════╪══════════╡
│ 7 = 0111  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 6 = 0110  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 5 = 0101  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 4 = 0100  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 3 = 0011  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 2 = 0010  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 1 = 0001  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 0 = 0000  

In order to search for non-negative outputs of a function, we use an oracle that matches outcomes that have 0 as the first digit in the value register:

In [5]:
from algo import oracle_match_0

oracle = oracle_match_0(n_key + n_value, n_key + n_value - 1)

Let's use this oracle to build the Grover operator (using the function `grover_circuit` from chapter 9) and apply one iteration:

In [6]:
from algo import grover_circuit

prepare = build_polynomial_circuit(n_key, n_value, terms)

qc = grover_circuit(prepare, oracle, 1)

In [7]:
grid_state(qc.run(), n_key, neg = True, show_probs = False)



╒═══════════╤══════════╤══════════╤══════════╤═══════════╕
│           │ 0 = 00   │ 1 = 01   │ 2 = 10   │ 3 = 11    │
╞═══════════╪══════════╪══════════╪══════════╪═══════════╡
│ 7 = 0111  │          │          │          │           │
├───────────┼──────────┼──────────┼──────────┼───────────┤
│ 6 = 0110  │          │          │          │           │
├───────────┼──────────┼──────────┼──────────┼───────────┤
│ 5 = 0101  │          │          │          │           │
├───────────┼──────────┼──────────┼──────────┼───────────┤
│ 4 = 0100  │          │          │          │           │
├───────────┼──────────┼──────────┼──────────┼───────────┤
│ 3 = 0011  │          │          │          │           │
├───────────┼──────────┼──────────┼──────────┼───────────┤
│ 2 = 0010  │          │          │          │           │
├───────────┼──────────┼──────────┼──────────┼───────────┤
│ 1 = 0001  │          │          │          │           │
├───────────┼──────────┼──────────┼──────────┼────────

If we apply two iterations, we get a state with all outputs having equal probability:

In [8]:
qc = grover_circuit(prepare, oracle, 2)

In [9]:
grid_state(qc.run(), n_key, neg = True, show_probs = False)



╒═══════════╤══════════╤══════════╤══════════╤══════════╕
│           │ 0 = 00   │ 1 = 01   │ 2 = 10   │ 3 = 11   │
╞═══════════╪══════════╪══════════╪══════════╪══════════╡
│ 7 = 0111  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 6 = 0110  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 5 = 0101  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 4 = 0100  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 3 = 0011  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 2 = 0010  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 1 = 0001  │          │          │          │          │
├───────────┼──────────┼──────────┼──────────┼──────────┤
│ 0 = 0000  

### Finding optimal outcomes with the Grover Optimizer (11.2)

In this example, we want to find the maximum of the function

$$
f(k) = -(k-3)^2 + 3
$$

where $0 \le k < 8$.

We will use $n = 3$ qubits for the key register to represent integer inputs $0 \le k < 8$, and $m = 6$ qubits for the value register to represent the outputs using Two's Complement:

In [10]:
n_key = 3
n_value = 6

The list of tuples for the terms of binary variables is:

In [11]:
terms = [(8, [2]), (8, [1]), (5, [0]), (-16, [1, 2]), (-8, [0, 2]), (-4, [0, 1]), (-6, [])]

We can check that these terms are correct using the function `poly`:

In [12]:
from util import poly

p = poly(n_key, terms, False)
f = lambda k: -(k - 3)**2 + 3

for k in range(len(p)):
    assert(p[k] == f(k))

Let's create the circuit that encodes the function $f$ in a quantum state using the variables defined above:

In [13]:
qc = build_polynomial_circuit(n_key, n_value, terms)

We will use the dictionary defined below to keep track of the flow state while progressing through the steps:

In [25]:
import copy

circuit_params = {'n_key': n_key, 'n_value': n_value, 'terms': terms}

flow_state = {
    'last_good_outcome_results': (None, -1),
    'failure_count': 0,
    'circuit_params': circuit_params,
    'initial_circuit_params': copy.deepcopy(circuit_params),
}

In [26]:
grid_state(qc.run(), n_key, neg = True, show_probs = False)



╒══════════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╕
│              │ 0 = 000   │ 1 = 001   │ 2 = 010   │ 3 = 011   │ 4 = 100   │ 5 = 101   │ 6 = 110   │ 7 = 111   │
╞══════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 31 = 011111  │           │           │           │           │           │           │           │           │
├──────────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 30 = 011110  │           │           │           │           │           │           │           │           │
├──────────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 29 = 011101  │           │           │           │           │           │           │           │           │
├──────────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼──────

We also need to define a stop condition. In this example, we will use the number of failures (`failure_count`) to define the stopping condition:

In [27]:
stopping_condition = lambda flow_state: flow_state['failure_count'] > 7

In order to search for non-negative values, we use an oracle that tags the outcomes with 0 in the first digit of the value register:

In [28]:
prepare = build_polynomial_circuit(n_key, n_value, terms)

oracle = oracle_match_0(n_key + n_value, n_key + n_value - 1)

Let's start with $r = 0$, meaning that no Grover iterations are applied.
We will simulate 100 measurements of this circuit:

In [29]:
qc = grover_circuit(prepare, oracle, 0)

shots = 100
result = qc.measure(shots = shots)

The most frequent measurement outcome is:

In [30]:
outcome = max(result['counts'].items(), key = lambda k: k[1])[0]

We can use the function defined below to get the input-output pair that this outcome corresponds to:

In [31]:
from util import padded_bin

def process_outcome(outcome, state):
    k = int(padded_bin(n_key + n_value, outcome)[n_value:], 2)
    v = int(padded_bin(n_key + n_value, outcome)[:n_value], 2)
    if v >= 2**(n_value - 1):
        v = v - 2**n_value
    v -= state['circuit_params']['terms'][0][0] - state['initial_circuit_params']['terms'][0][0]
    assert(v == p[k])
    return (k, v)

Note that we have an equal likelihood of getting any of the encoded input-output pairs at this step because we did not apply a Grover operator.


In [32]:
outcome_results = process_outcome(outcome, flow_state)

In [33]:
print(outcome_results)

(1, -1)


We will use the function `progress` to check if the outcome is better than the previous result:

In [34]:
def progress(results, state):
    return results[1] > state['last_good_outcome_results'][1] if state['last_good_outcome_results'][1] else True

In [35]:
progress(outcome_results, flow_state)

False

If `progress` returns false, we will increase the failure count variable as we failed to make progress. **Please note when you are running to notebook you may get a different result, please re-run the above code (starting with the cell where we define a circuit with 0 iterations) until progress is False to continue the example.**

In [36]:
flow_state['failure_count'] += 1

Now, we check if the stop condition is satisfied.

In [37]:
stopping_condition(flow_state)

False

The stop condition is not satisfied, so, we will perform $r = 1$ Grover iterations:

In [38]:
qc = grover_circuit(prepare, oracle, 1)

In [39]:
grid_state(qc.run(), n_key, neg = True, show_probs = False)



╒══════════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╤═══════════╕
│              │ 0 = 000   │ 1 = 001   │ 2 = 010   │ 3 = 011   │ 4 = 100   │ 5 = 101   │ 6 = 110   │ 7 = 111   │
╞══════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 31 = 011111  │           │           │           │           │           │           │           │           │
├──────────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 30 = 011110  │           │           │           │           │           │           │           │           │
├──────────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┤
│ 29 = 011101  │           │           │           │           │           │           │           │           │
├──────────────┼───────────┼───────────┼───────────┼───────────┼───────────┼───────────┼──────

In [40]:
qc = grover_circuit(prepare, oracle, 1)

Let's take 100 simulated measurements of this circuit:

In [41]:
shots = 100
result = qc.measure(shots = shots)

Again, we look at the input-output pair the most frequent outcome corresponds to:

In [42]:
outcome = max(result['counts'].items(), key = lambda k: k[1])[0]

outcome_results = process_outcome(outcome, flow_state)

In [43]:
outcome_results

(3, 3)

We will use the function `update_circuit_params` to update the parameters of the circuit $A$:

In [44]:
def update_circuit_params(outcome_results, flow_state):
    circuit_params = flow_state['circuit_params']
    k, v = outcome_results
    t = circuit_params['terms']
    t[0] = (t[0][0] - v - 1, [])
    print('\n------------------------')
    print('New free term:', t[0][0])

In [45]:
if progress(outcome_results, flow_state):
    update_circuit_params(outcome_results, flow_state)


------------------------
New free term: 4


We will use the function `build_circuit` to create a circuit with the updated parameters:

In [46]:
def build_circuit(flow_state):
    return build_polynomial_circuit(flow_state['circuit_params']['n_key'], flow_state['circuit_params']['n_value'], flow_state['circuit_params']['terms'])

Next, we encode the updated function and repeat the iteration schedule, starting with no Grover iterations:

In [47]:
prepare = build_circuit(flow_state)

oracle = oracle_match_0(n_key + n_value, n_key + n_value - 1)

qc = grover_circuit(prepare, oracle, 0)

shots = 100
result = qc.measure(shots = shots)

Now, let's check if we made progress towards finding the maximum:

In [48]:
outcome = max(result['counts'].items(), key = lambda k: k[1])[0]
outcome_results = process_outcome(outcome, flow_state)

In [49]:
outcome_results

(6, -6)

In [50]:
progress(outcome_results, flow_state)

False

The function `progress` returns `False` and we increase the failure count:

In [51]:
flow_state['failure_count'] += 1

Listing 11.1 Function that implements the Grover optimizer

In [52]:
def grover_optimizer(circuit_params,
                     build_circuit, oracle,
                     update_circuit_params, progress, process_outcome,
                     stopping_condition = lambda flow_state: flow_state['failure_count'] > 7,
                     schedule = [0, 1],
                     starting_result = (None, -1)):
    flow_state = {
        'last_good_outcome_results': starting_result,
        'failure_count': 0,
        'initial_circuit_params': copy.deepcopy(circuit_params),
        'circuit_params': circuit_params
    }

    shots = 100

    def update(outcome_results, flow_state):
        flow_state['last_good_outcome_results'] = outcome_results
        flow_state['failure_count'] = 0
        update_circuit_params(outcome_results, flow_state)

    done = False
    counter = 0
    while not done:
        counter += 1
        for r in schedule:
            print('\niteration', r)
            function = build_circuit(flow_state)
            qc = grover_circuit(function, oracle, r)

            result = qc.measure(shots = shots)

            flow_state['last_run_result'] = result

            outcome = max(result['counts'].items(), key = lambda k: k[1])[0]

            outcome_results = process_outcome(outcome, flow_state)

            if progress(outcome_results, flow_state):
                print('progress', outcome_results)

                update(outcome_results, flow_state)
                break
            else:
                flow_state['failure_count'] += 1
                print('failure', outcome_results)

                if stopping_condition(flow_state):
                    print('\nSTOPPING WITH OUTCOME RESULTS', flow_state['last_good_outcome_results'])
                    done = True
                    break

    return flow_state['last_good_outcome_results']

Let's use this function to solve our example problem:

In [53]:
def test_grover_optimizer():
    n_key = 3
    n_value = 6

    terms = ([(-6, [])] + [(6*2**k, [k]) for k in range(n_key)] +
             [(-2**(j+k), ([j, k] if j != k else [j]))  for j in range(n_key) for k in range(n_key)]) # squares

    p = poly(n_key, terms, False)
    f = lambda k: -(k - 3)**2 + 3 # -k**2 +6*k - 6
    for k in range(len(p)):
        assert(p[k] == f(k))

    def update_circuit_params(outcome_results, flow_state):
        circuit_params = flow_state['circuit_params']
        k, v = outcome_results
        t = circuit_params['terms']
        t[0] = (t[0][0] - v - 1, [])
        print('\n------------------------')
        print('New free term:', t[0][0])

    def progress(results, state):
        return results[1] > state['last_good_outcome_results'][1] if state['last_good_outcome_results'][1] else True

    def process_outcome(outcome, state):
        k = int(padded_bin(n_key + n_value, outcome)[n_value:], 2)
        v = int(padded_bin(n_key + n_value, outcome)[:n_value], 2)
        if v >= 2**(n_value - 1):
            v = v - 2**n_value
        v -= state['circuit_params']['terms'][0][0] - state['initial_circuit_params']['terms'][0][0]
        assert(v == p[k])
        return (k, v)

    oracle = oracle_match_0(n_key + n_value, n_key + n_value - 1)

    def build_circuit(flow_state):
        return build_polynomial_circuit(flow_state['circuit_params']['n_key'], flow_state['circuit_params']['n_value'], flow_state['circuit_params']['terms'])

    optimum = grover_optimizer({'n_key': n_key, 'n_value': n_value, 'terms': terms},
            build_circuit, oracle, update_circuit_params, progress, process_outcome)
    assert(optimum[1] == max(p))

In [54]:
test_grover_optimizer()


iteration 0
failure (0, -6)

iteration 1
progress (4, 2)

------------------------
New free term: -9

iteration 0
progress (3, 3)

------------------------
New free term: -13

iteration 0
failure (3, 3)

iteration 1
failure (4, 2)

iteration 0
failure (2, 2)

iteration 1
failure (2, 2)

iteration 0
failure (1, -1)

iteration 1
failure (3, 3)

iteration 0
failure (0, -6)

iteration 1
failure (0, -6)

STOPPING WITH OUTCOME RESULTS (3, 3)


### Solving the knapsack problem with a Grover optimizer (11.3)

**Preparing the state**

Let's define the size of each register:

In [55]:
n_key = 3
n_value = 3
n_weight = 4

We can use a list of tuples to represent these value and weight functions:

In [56]:
v = [(2, [0]), (3, [1]), (1, [2])]

w = [(3, [0]), (2, [1]), (1, [2])]

The function `build_knapsack_circuit` defined below creates a circuit with three registers (with the given sizes `n_key`, `n_value`, and `n_weight`) and encodes the given value and weight function (`v` and `w`, respectively).


In [57]:
from algo import encode_term

def build_knapsack_circuit(n_key, n_value, n_weight, v, w):

    key = QuantumRegister(n_key)
    value = QuantumRegister(n_value)
    weight = QuantumRegister(n_weight)
    circuit = QuantumCircuit(key, value, weight)

    for i in range(len(key)):
        circuit.h(key[i])

    for i in range(len(value)):
        circuit.h(value[i])

    for i in range(len(weight)):
        circuit.h(weight[i])

    for (coeff, vars) in v:
        encode_term(coeff, vars, circuit, key, value)

    circuit.iqft(value[::-1], swap=False)

    for (coeff, vars) in w:
        encode_term(coeff, vars, circuit, key, weight)

    circuit.iqft(weight[::-1], swap=False)

    return circuit

Let's create the starting circuit:

In [58]:
qc = build_knapsack_circuit(n_key, n_value, n_weight, v, w)

**Encoding constraints**

To encode the constraint that the weight of a selection is less than or equal to 4, we will adjust the encoded weight function to $w^\prime = 4 - w$, so that the integer encoded in the weight register will be non-negative for selections that meet the weight requirement ($w^\prime \ge 0$).
This way, our oracle will tag selections with a total weight of 4 or less.
We adjust the terms using the code below:

In [59]:
w = [(3, [0]), (2, [1]), (1, [2])]
max_weight = 4

w_adjusted = [(-item[0], item[1]) for item in w] + [(max_weight, [])]

Similarly, we adjust the encoded value function.
We can see that the most valuable item has a value of 3, and its weight is below the allowed capacity. This information helps us restrict the search to selections that have a combined value of at least 3.
We will shift down the values by 3 by adjusting the value function ($v^\prime = v - 3$) so that the integer encoded in the value register for the desired selections will be non-negative ($v^\prime \ge 0$).
We adjust the terms using the following code:

In [60]:
values = [2, 3, 1]
v = [(2, [0]), (3, [1]), (1, [2])]

v_adjusted = [(-max(values) - 0, [])] + v

**Defining the parameters of the Grover optimizer**

We can use the function `oracle_match_0_multi` defined in chapter 10 to create this oracle:

In [61]:
from algo import oracle_match_0_multi

oracle = oracle_match_0_multi(n_key + n_value + n_weight,
                                  [n_key + n_value - 1, n_key + n_value + n_weight - 1])

We will use the function `build_circuit` to create the circuit to encode the selections and their corresponding values and weights:

In [62]:
def build_circuit(flow_state):
    return build_knapsack_circuit(flow_state['circuit_params']['n_key'], flow_state['circuit_params']['n_value'], flow_state['circuit_params']['n_weight'],
                        flow_state['circuit_params']['v'], flow_state['circuit_params']['w'])

In [63]:
def get_selection_value(selection_binary, v):
    val = 0
    for i in range(len(selection_binary)):
        val += int(selection_binary[::-1][i]) * v[i][0]

    return val


def get_selection_weight(selection_binary, w):
    weight = 0
    for i in range(len(selection_binary)):
        weight += int(selection_binary[::-1][i]) * w[i][0]

    return weight

To interpret the measurement outcome at each step, we will use the function `process_outcome` below that takes the binary outcome and returns the selection that outcome corresponds to and the weight and value of the selection.

In [64]:
def process_outcome(outcome, flow_state):
    n = flow_state['circuit_params']['n_key'] + flow_state['circuit_params']['n_value'] + flow_state['circuit_params']['n_weight']
    outcome_selection = padded_bin(n, outcome)[-n_key:]
    outcome_value = get_selection_value(outcome_selection, v)
    outcome_weight = get_selection_weight(outcome_selection, w)
    return outcome_selection, outcome_value, outcome_weight

To check if there is progress we will use the function defined below:

In [65]:
def progress(results, state):
    outcome_selection, outcome_value, outcome_weight = results
    min_value = state['circuit_params']['min_value']
    return (outcome_value >= min_value) and (outcome_weight <= max_weight)

The function `update_circuit_params` shifts the value function to search for selections with values higher than the current best selection at a given step.
The function `action_on_progress` prints useful information about each step.

In [66]:
def action_on_progress(flow_state):
    n_key = flow_state['circuit_params']['n_key']
    n_value = flow_state['circuit_params']['n_value']
    n_weight = flow_state['circuit_params']['n_weight']
    s = flow_state['last_run_result']['state vector']
    # print(knapsack_table_new(s, n_key, n_value, n_weight))

    n = n_key + n_value + n_weight
    ss = sorted(enumerate(s), key=lambda x: int(padded_bin(n, x[0])[-flow_state['circuit_params']['n_key']:], 2))

    print(f'\nLooking for values >= ',{flow_state['circuit_params']['min_value']})

In [67]:
def update_circuit_params(outcome_results, flow_state):
    circuit_params = flow_state['circuit_params']
    outcome_selection, outcome_value, outcome_weight = outcome_results
    v = circuit_params['v']

    v[0] = (-outcome_value - 1, [])

    circuit_params['min_value'] = outcome_value + 1
    action_on_progress(flow_state)

Now we are ready to use the function `grover_optimizer` to solve the problem. Note that in the second parameter we include the minimum value to look for.

In [68]:
grover_optimizer({'n_key': n_key, 'n_value': n_value, 'n_weight': n_weight, 'v': v_adjusted, 'w': w_adjusted, 'min_value': 3},
                               build_circuit, oracle, update_circuit_params, progress, process_outcome,
                                stopping_condition = lambda flow_state: flow_state['failure_count'] > 3)


iteration 0
failure ('111', 6, 6)

iteration 1
progress ('110', 4, 3)

Looking for values >=  {5}

iteration 0
failure ('001', 2, 3)

iteration 1
failure ('000', 0, 0)

iteration 0
failure ('100', 1, 1)

iteration 1
failure ('000', 0, 0)

STOPPING WITH OUTCOME RESULTS ('110', 4, 3)


('110', 4, 3)